In [1]:
pip install --upgrade pillow

^C
Note: you may need to restart the kernel to use updated packages.


In [78]:
#version 0.1.7
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk

class PhotoSorterApp:
    def __init__(self, root):
        self.root = root
        self.root.title("АААААААААААаааааааааааа------А")

        self.source_dir = None
        self.destination_dirs = {"norki": None, "sori": None, "defect": None}
        self.current_index = 0
        self.image_files = []
        self.image_canvas = None
        self.photo_image = None
        self.scale = 1.0
        self.image_container = None
        self.image_item = None

        self.setup_ui()

    def setup_ui(self):
        
        button_frame = tk.Frame(self.root)
        button_frame.pack(fill=tk.X)

        self.source_button = tk.Button(button_frame, text="Select Source Directory", command=self.select_source_dir)
        self.source_button.pack(side=tk.LEFT)

        self.destination_buttons = {}
        for dir_name in self.destination_dirs.keys():
            button = tk.Button(button_frame, text=f"Select {dir_name.capitalize()} Directory", command=lambda dir=dir_name: self.select_destination_dir(dir))
            button.pack(side=tk.LEFT)
            self.destination_buttons[dir_name] = button
        
        self.image_canvas = tk.Canvas(self.root, cursor="fleur")
        self.image_canvas.pack(fill=tk.BOTH, expand=True)
        self.image_container = self.image_canvas.create_rectangle(0, 0, 1, 1)

        tk.Button(self.root, text="Previous", command=self.previous_image).pack(side=tk.LEFT)
        tk.Button(self.root, text="Next", command=self.next_image).pack(side=tk.RIGHT)

        for dir_name in self.destination_dirs.keys():
            tk.Button(self.root, text=f"Copy to {dir_name.capitalize()}", command=lambda dir=dir_name: self.copy_to_dir(dir)).pack()

        self.image_canvas.bind("<ButtonPress-1>", self.move_from)
        self.image_canvas.bind("<B1-Motion>", self.move_to)
        self.image_canvas.bind("<MouseWheel>", self.zoom)
        self.image_canvas.bind("<Button-4>", lambda event: self.zoom(event, delta=1.1))  
        self.image_canvas.bind("<Button-5>", lambda event: self.zoom(event, delta=0.9))  
        self.root.bind("<Left>", lambda event: self.previous_image())
        self.root.bind("<Right>", lambda event: self.next_image())
        self.root.bind("<Key-1>", lambda event: self.copy_to_dir("norki"))
        self.root.bind("<Key-2>", lambda event: self.copy_to_dir("sori"))
        self.root.bind("<Key-3>", lambda event: self.copy_to_dir("defect"))


    def select_source_dir(self):
        self.source_dir = filedialog.askdirectory(title="Select Source Directory")
        if self.source_dir:
            self.source_button.config(text="Selected")
            self.source_button.config(state=tk.DISABLED)
            self.load_images()

    def select_destination_dir(self, dir_name):
        self.destination_dirs[dir_name] = filedialog.askdirectory(title=f"Select {dir_name.capitalize()} Directory")
        if self.destination_dirs[dir_name]:
            self.destination_buttons[dir_name].config(text="Selected")
            self.destination_buttons[dir_name].config(state=tk.DISABLED)
            if not os.path.exists(self.destination_dirs[dir_name]):
                os.makedirs(self.destination_dirs[dir_name])

    def copy_to_dir(self, dir_name):
        if self.current_index < len(self.image_files) and self.destination_dirs[dir_name]:
            shutil.copy(os.path.join(self.source_dir, self.image_files[self.current_index]), self.destination_dirs[dir_name])
            self.next_image()

    def load_images(self):
        if self.source_dir and all(self.destination_dirs.values()):
            self.image_files = [f for f in os.listdir(self.source_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif'))]
            self.current_index = 0
            self.show_image()
        else:
            messagebox.showerror("Error", "Сначала выбери пути к сохранению")

    def show_image(self):
        if self.current_index < len(self.image_files):
            image_path = os.path.join(self.source_dir, self.image_files[self.current_index])
            with Image.open(image_path) as img:
                self.original_image = img.copy()  # Сохраняем исходный размер изображения
                img_width, img_height = img.size
                # Вычисляем новый размер изображения, учитывая текущий масштаб
                new_width = int(img_width * self.scale)
                new_height = int(img_height * self.scale)
                img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
                self.photo_image = ImageTk.PhotoImage(img)
                # Центрируем изображение в холсте
                self.image_canvas.delete(self.image_item)
                self.image_item = self.image_canvas.create_image(self.image_canvas.winfo_width() // 2, self.image_canvas.winfo_height() // 2, image=self.photo_image)
                # Масштабируем изображение относительно центра холста
                self.image_canvas.scale(self.image_item, self.image_canvas.winfo_width() // 2, self.image_canvas.winfo_height() // 2, self.scale, self.scale)


    def update_canvas_size(self):
        self.image_canvas.config(width=self.root.winfo_width(), height=self.root.winfo_height())

    def previous_image(self):
        if self.current_index > 0:
            self.current_index -= 1
            self.show_image()

    def next_image(self):
        if self.current_index < len(self.image_files) - 1:
            self.current_index += 1
            self.show_image()

    def move_from(self, event):
        self.image_canvas.scan_mark(event.x, event.y)

    def move_to(self, event):
        self.image_canvas.scan_dragto(event.x, event.y, gain=1)

    def zoom(self, event):
        if event.delta > 0:
            self.scale *= 1.5  
        elif event.delta < 0:
            self.scale *= 0.8  
        self.show_image()  # Обновление изображения с учетом нового масштаба


if __name__ == "__main__":
    root = tk.Tk()
    root.geometry("1200x800")
    app = PhotoSorterApp(root)
    root.mainloop()

In [116]:
#version 0.2.1
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk

class PhotoSorterApp:
    def __init__(self, root):
        self.root = root
        self.root.title("АААААААААААаааааааааааа------А")

        self.source_dir = None
        self.destination_dirs = {"norki": None, "sori": None, "defect": None}
        self.current_index = 0
        self.image_files = []
        self.image_canvas = None
        self.photo_image = None
        self.scale = 1.0
        self.image_container = None
        self.image_item = None
        self.destination_buttons = {}

        self.setup_ui()

    def setup_ui(self):
        
        # Создаем фрейм для кнопок сортировки и навигации
        navigation_frame = tk.Frame(self.root)
        navigation_frame.pack(fill=tk.X)

        # Кнопки навигации
        tk.Button(navigation_frame, text="Назад", command=self.previous_image).pack(side=tk.LEFT, padx=5)
        tk.Button(navigation_frame, text="Вперёд", command=self.next_image).pack(side=tk.LEFT, padx=5)

        # Кнопки сортировки
        sort_frame = tk.Frame(navigation_frame)
        sort_frame.pack(side=tk.RIGHT)

        for dir_name in self.destination_dirs.keys():
            button = tk.Button(sort_frame, text=f"Это {dir_name.capitalize()}", command=lambda dir=dir_name: self.copy_to_dir(dir))
            button.pack(side=tk.LEFT, padx=5)
            self.destination_buttons[dir_name] = button

            button_frame = tk.Frame(self.root)
            button_frame.pack(fill=tk.X)

            self.source_button = tk.Button(button_frame, text="Путь к фото", command=self.select_source_dir)
            self.source_button.pack(side=tk.LEFT)

            self.destination_buttons = {}
            for dir_name in self.destination_dirs.keys():
                button = tk.Button(button_frame, text=f"Выбери путь к {dir_name.capitalize()}", command=lambda dir=dir_name: self.select_destination_dir(dir))
                button.pack(side=tk.LEFT)
                self.destination_buttons[dir_name] = button
            
            self.image_canvas = tk.Canvas(self.root, cursor="fleur")
            self.image_canvas.pack(fill=tk.BOTH, expand=True)
            self.image_container = self.image_canvas.create_rectangle(0, 0, 1, 1)
            
            self.image_canvas.bind("<ButtonPress-1>", self.move_from)
            self.image_canvas.bind("<B1-Motion>", self.move_to)
            self.image_canvas.bind("<MouseWheel>", self.zoom)
            self.image_canvas.bind("<Button-4>", lambda event: self.zoom(event, delta=1.1))  
            self.image_canvas.bind("<Button-5>", lambda event: self.zoom(event, delta=0.9))  
            self.root.bind("<Left>", lambda event: self.previous_image())
            self.root.bind("<Right>", lambda event: self.next_image())
            self.root.bind("<Key-1>", lambda event: self.copy_to_dir("norki"))
            self.root.bind("<Key-2>", lambda event: self.copy_to_dir("sori"))
            self.root.bind("<Key-3>", lambda event: self.copy_to_dir("defect"))


    def select_source_dir(self):
        self.source_dir = filedialog.askdirectory(title="Путь к фото")
        if self.source_dir:
            self.source_button.config(text="Selected")
            self.load_images()

    def select_destination_dir(self, dir_name):
        self.destination_dirs[dir_name] = filedialog.askdirectory(title=f"Выбери путь к {dir_name.capitalize()}")
        if self.destination_dirs[dir_name]:
            self.destination_buttons[dir_name].config(text="Selected")
            if not os.path.exists(self.destination_dirs[dir_name]):
                os.makedirs(self.destination_dirs[dir_name])

    def copy_to_dir(self, dir_name):
        if self.current_index < len(self.image_files) and self.destination_dirs[dir_name]:
            shutil.copy(os.path.join(self.source_dir, self.image_files[self.current_index]), self.destination_dirs[dir_name])
            self.next_image()

    def load_images(self):
        if self.source_dir and all(self.destination_dirs.values()):
            self.image_files = [f for f in os.listdir(self.source_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif'))]
            self.current_index = 0
            self.show_image()
        else:
            messagebox.showerror("Error", "Сначала выбери пути к сохранению")

    def show_image(self):
        if self.current_index < len(self.image_files):
            image_path = os.path.join(self.source_dir, self.image_files[self.current_index])
            with Image.open(image_path) as img:
                self.original_image = img.copy()  # Сохраняем исходный размер изображения
                img_width, img_height = img.size
                # Вычисляем новый размер изображения, учитывая текущий масштаб
                new_width = int(img_width * self.scale)
                new_height = int(img_height * self.scale)
                img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
                self.photo_image = ImageTk.PhotoImage(img)
                # Центрируем изображение в холсте
                self.image_canvas.delete(self.image_item)
                self.image_item = self.image_canvas.create_image(self.image_canvas.winfo_width() // 2, self.image_canvas.winfo_height() // 2, image=self.photo_image)
                # Масштабируем изображение относительно центра холста
                self.image_canvas.scale(self.image_item, self.image_canvas.winfo_width() // 2, self.image_canvas.winfo_height() // 2, self.scale, self.scale)


    def update_canvas_size(self):
        self.image_canvas.config(width=self.root.winfo_width(), height=self.root.winfo_height())

    def previous_image(self):
        if self.current_index > 0:
            self.current_index -= 1
            self.show_image()

    def next_image(self):
        if self.current_index < len(self.image_files) - 1:
            self.current_index += 1
            self.show_image()

    def move_from(self, event):
        self.image_canvas.scan_mark(event.x, event.y)

    def move_to(self, event):
        self.image_canvas.scan_dragto(event.x, event.y, gain=1)

    def zoom(self, event):
        if event.delta > 0:
            self.scale *= 1.5  
        elif event.delta < 0:
            self.scale *= 0.8  
        self.show_image()  # Обновление изображения с учетом нового масштаба


if __name__ == "__main__":
    root = tk.Tk()
    root.geometry("1200x800")
    app = PhotoSorterApp(root)
    root.mainloop()

In [19]:
#version 0.2.1
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk

class PhotoSorterApp:
    def __init__(self, root):
        self.root = root
        self.root.title("АААААААААААаааааааааааа------А")

        self.source_dir = None
        self.destination_dirs = {"norki": None, "sori": None, "defect": None}
        self.current_index = 0
        self.image_files = []
        self.image_canvas = None
        self.photo_image = None
        self.scale = 1.0
        self.image_container = None
        self.image_item = None
        self.destination_buttons = {}
        self.image_label = None

        self.setup_ui()

    def setup_ui(self):
        
        # фрейм кнопок
        navigation_frame = tk.Frame(self.root)
        navigation_frame.pack(fill=tk.X)

        # батон навигации
        tk.Button(navigation_frame, text="Назад", command=self.previous_image).pack(side=tk.LEFT, padx=5)
        tk.Button(navigation_frame, text="Вперёд", command=self.next_image).pack(side=tk.LEFT, padx=5)

        # метка фото
        self.image_label = tk.Label(navigation_frame, text="")
        self.image_label.pack(side=tk.LEFT, padx=5)
        
        # батон сорт
        sort_frame = tk.Frame(navigation_frame)
        sort_frame.pack(side=tk.RIGHT)

        self.file_name_entry = tk.Entry(navigation_frame)
        self.file_name_entry.pack(side=tk.LEFT, padx=5)

        tk.Button(navigation_frame, text="Перейти к файлу", command=self.go_to_file).pack(side=tk.LEFT, padx=5)

        for dir_name in self.destination_dirs.keys():
            button = tk.Button(sort_frame, text=f"Это {dir_name.capitalize()}", command=lambda dir=dir_name: self.copy_to_dir(dir))
            button.pack(side=tk.LEFT, padx=5)
        self.destination_buttons[dir_name] = button

        button_frame = tk.Frame(self.root)
        button_frame.pack(fill=tk.X)

        self.source_button = tk.Button(button_frame, text="Путь к фото", command=self.select_source_dir)
        self.source_button.pack(side=tk.LEFT)

        self.destination_buttons = {}

        for dir_name in self.destination_dirs.keys():
            button = tk.Button(button_frame, text=f"Выбери путь к {dir_name.capitalize()}", command=lambda dir=dir_name: self.select_destination_dir(dir))
            button.pack(side=tk.LEFT)
            self.destination_buttons[dir_name] = button
        
        self.image_canvas = tk.Canvas(self.root, cursor="fleur")
        self.image_canvas.pack(fill=tk.BOTH, expand=True)
        self.image_container = self.image_canvas.create_rectangle(0, 0, 1, 1)
        
        self.image_canvas.bind("<ButtonPress-1>", self.move_from)
        self.image_canvas.bind("<B1-Motion>", self.move_to)
        self.image_canvas.bind("<MouseWheel>", self.zoom)
        self.image_canvas.bind("<Button-4>", lambda event: self.zoom(event, delta=1.1))  
        self.image_canvas.bind("<Button-5>", lambda event: self.zoom(event, delta=0.9))  
        self.root.bind("<Left>", lambda event: self.previous_image())
        self.root.bind("<Right>", lambda event: self.next_image())
        self.root.bind("<Key-1>", lambda event: self.copy_to_dir("norki"))
        self.root.bind("<Key-2>", lambda event: self.copy_to_dir("sori"))
        self.root.bind("<Key-3>", lambda event: self.copy_to_dir("defect"))
        self.file_name_entry.bind("<Button-1>", self.activate_search_field)
        self.file_name_entry.bind("<Button-1>", self.activate_search_field)
        self.file_name_entry.bind("<Return>", lambda event: self.go_to_file())

    def select_source_dir(self):
        self.source_dir = filedialog.askdirectory(title="Путь к фото")
        if self.source_dir:
            self.source_button.config(text="Selected")
            self.load_images()

    def select_destination_dir(self, dir_name):
        self.destination_dirs[dir_name] = filedialog.askdirectory(title=f"Выбери путь к {dir_name.capitalize()}")
        if self.destination_dirs[dir_name]:
            self.destination_buttons[dir_name].config(text="Selected")
            if not os.path.exists(self.destination_dirs[dir_name]):
                os.makedirs(self.destination_dirs[dir_name])

    def copy_to_dir(self, dir_name):
        if self.current_index < len(self.image_files) and self.destination_dirs[dir_name]:
            source_path = os.path.join(self.source_dir, self.image_files[self.current_index])
            destination_path = os.path.join(self.destination_dirs[dir_name], self.image_files[self.current_index])
            shutil.copy(source_path, destination_path)
            self.next_image()

    def load_images(self):
        if self.source_dir and all(self.destination_dirs.values()):
            self.image_files = sorted([f for f in os.listdir(self.source_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif'))], key=lambda x: int(os.path.splitext(x)[0]))
            self.current_index = 0
            self.show_image()
        else:
            messagebox.showerror("Error", "Сначала выбери пути к сохранению")

    def show_image(self):
        if self.current_index < len(self.image_files):
            image_path = os.path.join(self.source_dir, self.image_files[self.current_index])
            with Image.open(image_path) as img:
                self.original_image = img.copy()  # Сохраняем исходный размер изображения
                img_width, img_height = img.size
                # Вычисляем новый размер изображения, учитывая текущий масштаб
                new_width = int(img_width * self.scale)
                new_height = int(img_height * self.scale)
                img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
                self.photo_image = ImageTk.PhotoImage(img)
                # Центрируем изображение в холсте масштабируем относительно центра холста
                self.image_canvas.delete(self.image_item)
                self.image_item = self.image_canvas.create_image(self.image_canvas.winfo_width() // 2, self.image_canvas.winfo_height() // 2, image=self.photo_image)
                
                self.image_canvas.scale(self.image_item, self.image_canvas.winfo_width() // 2, self.image_canvas.winfo_height() // 2, self.scale, self.scale)
                self.image_label.config(text=self.image_files[self.current_index])

    def update_canvas_size(self):
        self.image_canvas.config(width=self.root.winfo_width(), height=self.root.winfo_height())

    def previous_image(self):
        if self.current_index > 0:
            self.current_index -= 1
            self.show_image()

    def next_image(self):
        if self.current_index < len(self.image_files) - 1:
            self.current_index += 1
            self.show_image()

    def move_from(self, event):
        self.image_canvas.scan_mark(event.x, event.y)

    def move_to(self, event):
        self.image_canvas.scan_dragto(event.x, event.y, gain=1)

    def zoom(self, event):
        if event.delta > 0:
            self.scale *= 1.5  
        elif event.delta < 0:
            self.scale *= 0.8  
        self.show_image() 

    def go_to_file(self):
        self.root.unbind("<Key-1>")
        self.root.unbind("<Key-2>")
        self.root.unbind("<Key-3>")

        file_name_without_ext = self.file_name_entry.get()
        if file_name_without_ext:
            for i, file_name in enumerate(self.image_files):
                if os.path.splitext(file_name)[0] == file_name_without_ext:
                    self.current_index = i
                    self.show_image()
                    break
            else:
                messagebox.showerror("Error", "Такой нету")

        self.file_name_entry.configure(state='disabled')

        self.root.bind("<Key-1>", lambda event: self.copy_to_dir("norki"))
        self.root.bind("<Key-2>", lambda event: self.copy_to_dir("sori"))
        self.root.bind("<Key-3>", lambda event: self.copy_to_dir("defect"))

    def activate_search_field(self, event):
        self.file_name_entry.configure(state='normal')
        self.file_name_entry.delete(0, tk.END)



if __name__ == "__main__":
    root = tk.Tk()
    root.geometry("1200x800")
    app = PhotoSorterApp(root)
    app.activate_search_field(None)
    root.mainloop()

In [26]:
#version 0.2.1
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk

class PhotoSorterApp:
    def __init__(self, root):
        self.root = root
        self.root.title("АААААААААААаааааааааааа------А")

        self.source_dir = None
        self.destination_dirs = {"norki": None, "sori": None, "defect": None}
        self.current_index = 0
        self.image_files = []
        self.image_canvas = None
        self.photo_image = None
        self.scale = 1.0
        self.image_container = None
        self.image_item = None
        self.destination_buttons = {}
        self.image_label = None
        self.bindings_active = False  # Флаг для отслеживания активности бинд

        self.setup_ui()

    def setup_ui(self):
        
        # фрейм кнопок
        navigation_frame = tk.Frame(self.root)
        navigation_frame.pack(fill=tk.X)

        # батон навигации
        tk.Button(navigation_frame, text="Назад", command=self.previous_image).pack(side=tk.LEFT, padx=5)
        tk.Button(navigation_frame, text="Вперёд", command=self.next_image).pack(side=tk.LEFT, padx=5)

        # метка фото
        self.image_label = tk.Label(navigation_frame, text="")
        self.image_label.pack(side=tk.LEFT, padx=5)
        
        # бат сорт
        sort_frame = tk.Frame(navigation_frame)
        sort_frame.pack(side=tk.RIGHT)

        self.file_name_entry = tk.Entry(navigation_frame)
        self.file_name_entry.pack(side=tk.LEFT, padx=5)

        tk.Button(navigation_frame, text="Перейти к файлу", command=self.go_to_file).pack(side=tk.LEFT, padx=5)

        for dir_name in self.destination_dirs.keys():
            button = tk.Button(sort_frame, text=f"Это {dir_name.capitalize()}", command=lambda dir=dir_name: self.copy_to_dir(dir))
            button.pack(side=tk.LEFT, padx=5)
        self.destination_buttons[dir_name] = button

        button_frame = tk.Frame(self.root)
        button_frame.pack(fill=tk.X)

        self.source_button = tk.Button(button_frame, text="Путь к фото", command=self.select_source_dir)
        self.source_button.pack(side=tk.LEFT)

        self.destination_buttons = {}

        for dir_name in self.destination_dirs.keys():
            button = tk.Button(button_frame, text=f"Выбери путь к {dir_name.capitalize()}", command=lambda dir=dir_name: self.select_destination_dir(dir))
            button.pack(side=tk.LEFT)
            self.destination_buttons[dir_name] = button
        
        self.image_canvas = tk.Canvas(self.root, cursor="fleur")
        self.image_canvas.pack(fill=tk.BOTH, expand=True)
        self.image_container = self.image_canvas.create_rectangle(0, 0, 1, 1)
        
        self.image_canvas.bind("<ButtonPress-1>", self.move_from)
        self.image_canvas.bind("<B1-Motion>", self.move_to)
        self.image_canvas.bind("<MouseWheel>", self.zoom)
        self.image_canvas.bind("<Button-4>", lambda event: self.zoom(event, delta=1.1))  
        self.image_canvas.bind("<Button-5>", lambda event: self.zoom(event, delta=0.9))  
        self.root.bind("<Left>", lambda event: self.previous_image())
        self.root.bind("<Right>", lambda event: self.next_image())
        self.root.bind("<Key-1>", lambda event: self.copy_to_dir("norki"))
        self.root.bind("<Key-2>", lambda event: self.copy_to_dir("sori"))
        self.root.bind("<Key-3>", lambda event: self.copy_to_dir("defect"))
        self.file_name_entry.bind("<Button-1>", self.activate_search_field)
        self.file_name_entry.bind("<Return>", lambda event: self.go_to_file())
        self.file_name_entry.bind("<FocusIn>", self.activate_search_field)
        self.file_name_entry.bind("<FocusOut>", self.deactivate_search_field)

    def select_source_dir(self):
        self.source_dir = filedialog.askdirectory(title="Путь к фото")
        if self.source_dir:
            self.source_button.config(text="Selected")
            self.load_images()

    def select_destination_dir(self, dir_name):
        self.destination_dirs[dir_name] = filedialog.askdirectory(title=f"Выбери путь к {dir_name.capitalize()}")
        if self.destination_dirs[dir_name]:
            self.destination_buttons[dir_name].config(text="Selected")
            if not os.path.exists(self.destination_dirs[dir_name]):
                os.makedirs(self.destination_dirs[dir_name])

    def copy_to_dir(self, dir_name):
        if self.current_index < len(self.image_files) and self.destination_dirs[dir_name]:
            source_path = os.path.join(self.source_dir, self.image_files[self.current_index])
            destination_path = os.path.join(self.destination_dirs[dir_name], self.image_files[self.current_index])
            shutil.copy(source_path, destination_path)
            self.next_image()

    def load_images(self):
        if self.source_dir and all(self.destination_dirs.values()):
            self.image_files = sorted([f for f in os.listdir(self.source_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif'))], key=lambda x: int(os.path.splitext(x)[0]))
            self.current_index = 0
            self.show_image()
        else:
            messagebox.showerror("Error", "Сначала выбери пути к сохранению")

    def show_image(self):
        if self.current_index < len(self.image_files):
            image_path = os.path.join(self.source_dir, self.image_files[self.current_index])
            with Image.open(image_path) as img:
                self.original_image = img.copy()  # Сохраняем исходный размер изображения
                img_width, img_height = img.size
                # Вычисляем новый размер изображения, учитывая текущий масштаб
                new_width = int(img_width * self.scale)
                new_height = int(img_height * self.scale)
                img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
                self.photo_image = ImageTk.PhotoImage(img)
                # Центрируем изображение в холсте масштабируем относительно центра холста
                self.image_canvas.delete(self.image_item)
                self.image_item = self.image_canvas.create_image(self.image_canvas.winfo_width() // 2, self.image_canvas.winfo_height() // 2, image=self.photo_image)
                
                self.image_canvas.scale(self.image_item, self.image_canvas.winfo_width() // 2, self.image_canvas.winfo_height() // 2, self.scale, self.scale)
                self.image_label.config(text=self.image_files[self.current_index])

    def update_canvas_size(self):
        self.image_canvas.config(width=self.root.winfo_width(), height=self.root.winfo_height())

    def previous_image(self):
        if self.current_index > 0:
            self.current_index -= 1
            self.show_image()

    def next_image(self):
        if self.current_index < len(self.image_files) - 1:
            self.current_index += 1
            self.show_image()

    def move_from(self, event):
        self.image_canvas.scan_mark(event.x, event.y)

    def move_to(self, event):
        self.image_canvas.scan_dragto(event.x, event.y, gain=1)

    def zoom(self, event):
        if event.delta > 0:
            self.scale *= 1.5  
        elif event.delta < 0:
            self.scale *= 0.8  
        self.show_image() 

    def go_to_file(self):
        self.unbind_keys()


        file_name_without_ext = self.file_name_entry.get()
        if file_name_without_ext:
            for i, file_name in enumerate(self.image_files):
                if os.path.splitext(file_name)[0] == file_name_without_ext:
                    self.current_index = i
                    self.show_image()
                    break
            else:
                messagebox.showerror("Error", "Такой нету")

        self.file_name_entry.configure(state='disabled')
        
        self.activate_keys()


    def activate_search_field(self, event):
        self.file_name_entry.configure(state='normal')
        self.file_name_entry.delete(0, tk.END)
        if self.bindings_active:
            self.unbind_keys()

    def activate_search_field(self, event):
        self.file_name_entry.configure(state='normal')
        self.file_name_entry.delete(0, tk.END)
        self.unbind_keys()  # Отключаем бинды

    def deactivate_search_field(self, event):
        self.file_name_entry.configure(state='disabled')
        if not self.bindings_active:
            self.bind_keys()

    def activate_keys(self):
        if not self.bindings_active:
            self.bind_keys()
            self.bindings_active = True

    def unbind_keys(self):
        self.root.unbind("<Key-1>")
        self.root.unbind("<Key-2>")
        self.root.unbind("<Key-3>")
        self.bindings_active = False


    def bind_keys(self):
        self.root.bind("<Key-1>", lambda event: self.copy_to_dir("norki"))
        self.root.bind("<Key-2>", lambda event: self.copy_to_dir("sori"))
        self.root.bind("<Key-3>", lambda event: self.copy_to_dir("defect"))
        self.bindings_active = True


if __name__ == "__main__":
    root = tk.Tk()
    root.geometry("1200x800")
    app = PhotoSorterApp(root)
    app.activate_search_field(None)
    root.mainloop()

In [28]:
!pyinstaller --onefile --noconsole --name "Sortiruemsya_2.1" sortphoto.py

712 INFO: PyInstaller: 6.6.0, contrib hooks: 2024.5
712 INFO: Python: 3.8.19 (conda)
727 INFO: Platform: Windows-10-10.0.19045-SP0
727 INFO: wrote c:\Users\Grishkov.DS\AAaaaa\Sortiruemsya_2.1.spec
729 INFO: Extending PYTHONPATH with paths
['c:\\Users\\Grishkov.DS\\AAaaaa']
950 INFO: checking Analysis
961 INFO: Building because C:\Users\Grishkov.DS\AAaaaa\sortphoto.py changed
961 INFO: Running Analysis Analysis-00.toc
961 INFO: Target bytecode optimization level: 0
961 INFO: Initializing module dependency graph...
961 INFO: Caching module graph hooks...
972 INFO: Analyzing base_library.zip ...
2558 INFO: Loading module hook 'hook-heapq.py' from 'c:\\Users\\Grishkov.DS\\.conda\\envs\\contorch\\lib\\site-packages\\PyInstaller\\hooks'...
2605 INFO: Loading module hook 'hook-encodings.py' from 'c:\\Users\\Grishkov.DS\\.conda\\envs\\contorch\\lib\\site-packages\\PyInstaller\\hooks'...
3983 INFO: Loading module hook 'hook-pickle.py' from 'c:\\Users\\Grishkov.DS\\.conda\\envs\\contorch\\lib\\s